# DX 704 Week 7 Project

This week's project will investigate issues in a quadcopter controller based using a linear quadratic regulator.
You will start with an existing model of the system and logs from a quadcopter based on it, investigate discrepancies, and ultimately train a new model of the system dynamics.

The full project description and a template notebook are available on GitHub: [Project 7 Materials](https://github.com/bu-cds-dx704/dx704-project-07).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Introduction

You've just joined a drone startup.
On your first day, you join your new team to watch a test flight for a new quadcopter prototype.
Watching the prototype fly, the team comments that it is not as smooth as usual and suspects that something is off in the controller.
They provide you a copy of the dynamics model and log data from the prototype to investigate.

The quadcopter control model is a slightly more sophisticated version of the 1D quadcopter problem previously considered.

The state vector $\mathbf{x}_t$ now includes an acceleration component, and the action vector now has a servo control for the throttle instead of a raw force component.
\begin{array}{rcl}
\mathbf{x}_t & = & \begin{bmatrix} x_t \\ v_t \\ a_t \end{bmatrix} \\
\mathbf{u_t} & = & \begin{bmatrix} u_t \end{bmatrix}
\end{array}

## Part 1: Reconstruct the Control Matrix

You are provided the dynamics model in the files `model-A.tsv`, `model-B.tsv`, `cost-Q.tsv` and `cost-R.tsv`.
Recompute the control matrix $\mathbf{K}$ to be used in the infinite horizon linear quadratic regulator problem.

In [1]:
# YOUR CHANGES HERE
import numpy as np
import pandas as pd

# ---- Load labeled TSVs into numpy arrays ----
def load_matrix(path):
    return pd.read_csv(path, sep="\t", index_col=0).astype(float)

A_df = load_matrix("model-A.tsv")
B_df = load_matrix("model-B.tsv")
Q_df = load_matrix("cost-Q.tsv")
R_df = load_matrix("cost-R.tsv")

A = A_df.values
B = B_df.values
Q = Q_df.values
R = R_df.values

# ---- Discrete-time LQR via Riccati iteration ----
def dlqr(A, B, Q, R, max_iters=10000, tol=1e-12):
    P = Q.copy()
    for _ in range(max_iters):
        BtP = B.T @ P
        S = R + BtP @ B                 # R + B' P B
        K = np.linalg.solve(S, BtP @ A) # (R + B' P B)^-1 B' P A
        P_next = A.T @ P @ A - A.T @ P @ B @ K + Q
        if np.linalg.norm(P_next - P, ord='fro') < tol:
            P = P_next
            break
        P = P_next
    # final K (in case we exited on tol)
    BtP = B.T @ P
    K = np.linalg.solve(R + BtP @ B, BtP @ A)
    return K, P

K, P = dlqr(A, B, Q, R)

# ---- Print and save ----
print("K row (u = -K x):")
print(K)

# Save with state labels
K_df = pd.DataFrame(K, columns=A_df.columns, index=["u"])
K_df.to_csv("control-K.tsv", sep="\t")

# (Optional) quick stability check
eigvals = np.linalg.eigvals(A - B @ K)
print("Closed-loop eigenvalues:", eigvals)

...

K row (u = -K x):
[[0.33445985 1.30445413 1.85813088]]
Closed-loop eigenvalues: [0.36935751+0.j         0.3862558 +0.39200236j 0.3862558 -0.39200236j]


Ellipsis

Save $\mathbf{K}$ in a file "control-K-intended.tsv" with columns x, v and a.

In [2]:
# YOUR CHANGES HERE


import numpy as np, pandas as pd

# Load labeled TSVs → numeric arrays
A = pd.read_csv("model-A.tsv", sep="\t", index_col=0).astype(float).values
B = pd.read_csv("model-B.tsv", sep="\t", index_col=0).astype(float).values
Q = pd.read_csv("cost-Q.tsv",  sep="\t", index_col=0).astype(float).values
R = pd.read_csv("cost-R.tsv",  sep="\t", index_col=0).astype(float).values

# Tiny discrete-time LQR (Riccati iteration)
P = Q.copy()
for _ in range(10000):
    BtP = B.T @ P
    K = np.linalg.solve(R + BtP @ B, BtP @ A)              # (R+B'PB)^-1 B' P A
    Pn = A.T @ P @ A - A.T @ P @ B @ K + Q                 # Riccati update
    if np.linalg.norm(Pn - P, 'fro') < 1e-12: break
    P = Pn
K = np.linalg.solve(R + (B.T @ P) @ B, (B.T @ P) @ A)      # final K

# Save exactly as requested
pd.DataFrame(K, columns=["x","v","a"], index=["u"]).to_csv("control-K-intended.tsv", sep="\t")

# (Optional) quick check
print("K (u = -Kx):\n", K)
print("Closed-loop eigenvalues:", np.linalg.eigvals(A - B @ K))

...

K (u = -Kx):
 [[0.33445985 1.30445413 1.85813088]]
Closed-loop eigenvalues: [0.36935751+0.j         0.3862558 +0.39200236j 0.3862558 -0.39200236j]


Ellipsis

Submit "control-K-intended.tsv" in Gradescope.

## Part 2: Recompute the Actions for the Logged States

You get access to the log data for the prototype as the file "qc-log.tsv".
It conveniently saves all the state and actions made.
Recompute the actions based on your $\mathbf{K}$ matrix computed in part 1.

In [4]:
# YOUR CHANGES HERE
import pandas as pd
import numpy as np

# ---- Load data ----
log = pd.read_csv("qc-log.tsv", sep="\t")     # contains logged states x, v, a
K = pd.read_csv("control-K-intended.tsv", sep="\t", index_col=0).values[0]  # load K row

# ---- Compute actions ----
X = log[["x", "v", "a"]].values
u_check = -X @ K                              # u = -Kx

# ---- Save results ----
out = pd.DataFrame({"index": log.index, "u_check": u_check})
out.to_csv("qc-check.tsv", sep="\t", index=False)

print("Saved qc-check.tsv with columns 'index' and 'u_check'")

...

Saved qc-check.tsv with columns 'index' and 'u_check'


Ellipsis

Save your computed actions as "qc-check.tsv" with columns "index" and "u_check".

In [ ]:
# YOUR CHANGES HERE

...

Submit "qc-check.tsv" in Gradescope.

## Part 3: Reverse Engineer the Actual Control Matrix

Now that you have found a systematic difference between your computed actions and the logged actions, estimate $
$, the control matrix that was actually used to choose actions in the prototype.

Hint: With a linear quadratic regulator, the optimal actions are always linear combinations of the state that are calculated using the control matrix.
You can use linear regression to reverse-engineer the coefficients in the control matrix.

In [5]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# ---- Load logged data ----
log = pd.read_csv("qc-log.tsv", sep="\t")     # must have columns: x, v, a, u

# ---- Prepare data for regression ----
X = log[["x", "v", "a"]].values
y = log["u"].values

# ---- Fit linear regression: u = -K_actual x ----
reg = LinearRegression(fit_intercept=False)
reg.fit(X, y)

K_actual = -reg.coef_    # negative sign because LQR uses u = -Kx

# ---- Save same format as intended ----
K_df = pd.DataFrame([K_actual], columns=["x", "v", "a"], index=["u"])
K_df.to_csv("control-K-actual.tsv", sep="\t")

print("Saved control-K-actual.tsv")
print(K_df)

...

Saved control-K-actual.tsv
          x        v         a
u  0.340438  1.30012  1.950117


Ellipsis

Save $\mathbf{K}_{\mathrm{actual}}$ in "control-K-actual.tsv" with the same format as "control-K-intended.tsv".

In [ ]:
# YOUR CHANGES HERE

...

Submit "actual-K.tsv" in Gradescope.

## Part 4: Recompute the System Dynamics from the Log Data

On further investigation, it turns out that the control matrix $\mathbf{K}$ in the prototype was modified to compensate for state prediction errors.
You would like to recompute the $\mathbf{A}$ and $\mathbf{B}$ matrices using the log data since they are used to predict the next states.
However, since the action vector $\mathbf{u}_t$ is linearly dependent on the state via $\mathbf{u}_t=-\mathbf{K} \mathbf{x}_t$, you need a new data set so you can separate the effects of the $\mathbf{A}$ and $\mathbf{B}$ matrices.
Your co-workers kindly provide a new file "qc-train.tsv" where noise was added to each action.
Estimate the true $\mathbf{A}$ and $\mathbf{B}$ matrices based on this file.

In [ ]:
# YOUR CHANGES HERE
import numpy as np
import pandas as pd

# ---------- Load training data ----------
df = pd.read_csv("qc-train.tsv", sep="\t")

# If your file already has explicit next-state columns like x_next/v_next/a_next,
# uncomment the next three lines and comment out the shift block below.
# Y = df[["x_next", "v_next", "a_next"]].values
# X = df[["x", "v", "a"]].values
# U = df[["u"]].values

# Build next-state by shifting (robust default)
df["x_next"] = df["x"].shift(-1)
df["v_next"] = df["v"].shift(-1)
df["a_next"] = df["a"].shift(-1)
df = df.dropna().reset_index(drop=True)

X = df[["x", "v", "a"]].values          # (N, 3)
U = df[["u"]].values                    # (N, 1)
Y = df[["x_next", "v_next", "a_next"]].values  # (N, 3)

# ---------- Least-squares: [A | B] from Y ≈ [X U] @ Theta ----------
Z = np.hstack([X, U])                   # (N, 4)
Theta, *_ = np.linalg.lstsq(Z, Y, rcond=None)  # Theta shape (4, 3)

A_new = Theta[:3, :]                    # (3, 3)  maps x_t → x_{t+1}
B_new = Theta[3:, :].T                  # (3, 1)  maps u_t → x_{t+1}; transpose to (3,1)

# ---------- Save with labels ----------
A_df = pd.DataFrame(A_new, index=["x","v","a"], columns=["x","v","a"])
B_df = pd.DataFrame(B_new, index=["x","v","a"], columns=["u"])

A_df.to_csv("model-A-new.tsv", sep="\t")
B_df.to_csv("model-B-new.tsv", sep="\t")


...

A_new:
               x             v             a
x  1.000000e+00 -1.281955e-16 -9.313780e-17
v  1.100000e+00  9.000000e-01  5.551115e-16
a  2.886580e-15  9.500000e-01  1.100000e+00

B_new:
               u
x -1.110223e-16
v -1.000000e-02
a  9.000000e-01


Ellipsis

Save $\mathbf{A}$ and $\mathbf{B}$ to "model-A-new.tsv" and "model-B-new.tsv" respectively.

In [ ]:
# YOUR CHANGES HERE

...

Submit "model-A-new.tsv" and "model-B-new.tsv" in Gradescope.

## Part 5: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 6: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.